In [1]:
"""
@author: amirjaved
"""
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Lambda,Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
import cv2
import matplotlib.image as mpimg
#helper class to define input shape and generate training images given image paths & steering angles
from utils import INPUT_SHAPE, batch_generator
np.random.seed(0)


#### configuring the gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)



1 Physical GPUs, 1 Logical GPUs


In [2]:
def preprocess(args,image):
#    image = image[60:-25, :, :]
    ##### args["input_shape"] is a tupple (height,width,channels)
    image = cv2.resize(image, (INPUT_SHAPE[1], INPUT_SHAPE[0]), cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    return image


In [3]:
def load_data(params):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable

    #read log data file and renaming columns
    data_df = pd.read_csv(os.path.join(params['data_dir'], 'data.csv'), sep=',')
    data_df = data_df.rename(columns=({'Unnamed: 0':'image_path','dist_0':'distance','ABS_RX':'steering_angle','ABS_Y':'speed'}))

    #we'll store the camera images as our input data
    X = data_df[['image_path']].values
    #and our steering commands as our output data
    y = data_df['steering_angle'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=params['test_size'], random_state=0)

    return X_train, X_test, y_train, y_test

In [4]:
def build_model(params):
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu',  input_shape=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', input_shape=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', input_shape=(2, 2)))
    #model.add(Conv2D(64, 3, 3, activation='elu'))
    #model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(params['keep_prob']))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model

In [5]:
params = {
    'data_dir': 'data/',
    'test_size': 0.2,
    'keep_prob': 0.5,
    'nb_epoch':10,
    'samples_per_epoch':2000,
    'batch_size': 40,
    'save_best_only':'true',
    'learning_rate': 1.0e-4
}
#print parameters
print('-' * 30)
print('Parameters')
print('-' * 30)
for key, value in params.items():
    print('{:<20} := {}'.format(key, value))
print('-' * 30)

------------------------------
Parameters
------------------------------
data_dir             := data/
test_size            := 0.2
keep_prob            := 0.5
nb_epoch             := 10
samples_per_epoch    := 2000
batch_size           := 40
save_best_only       := true
learning_rate        := 0.0001
------------------------------


In [12]:
#load data
X_train, X_test, y_train, y_test = load_data(params)
#build nvdia model
model = build_model(params)


### loading pretrained nvdia model
relative_path = "/trained_model_weights/model-005.h5" 
abs_path = os.path.abspath(os.getcwd()+relative_path)
model.load_weights(abs_path)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=params["learning_rate"]))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 128, 24)       1824      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 25, 36)        21636     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 5, 48)          43248     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 5, 48)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 720)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

In [21]:
prediction = []
for i in range(len(X_test)):
    img_path = X_test[i][0]
    image = mpimg.imread(params['data_dir']+str(img_path)+".jpg") 
    image = preprocess(params,image)
    pred = model.predict(np.expand_dims(image, axis=0), batch_size = 1)
    print([y_test[i]],pred[0])
    prediction.append(pred)
prediction = np.array(prediction)
np.savetxt("predictions.txt",prediction, delimiter=",")
#score = model.evaluate(data[""], Y, verbose=0)

[0.3] [-0.8690806]
[0.1] [-0.17692964]
[-1.0] [-0.4095557]
[0.5] [-1.1884676]
[-0.1] [-0.17900304]
[0.1] [0.10818715]
[-1.0] [-0.97459805]
[-0.0] [-0.01420544]
[-1.0] [-0.99520373]
[0.0] [-0.24253158]
[0.6] [-0.52934515]
[-0.7] [-1.0387157]
[-1.0] [-0.451948]
[0.1] [-0.3122819]
[0.0] [-0.8340818]
[-0.6] [-0.58229995]
[-0.2] [-1.1992201]
[-0.9] [-0.855568]
[-0.6] [-1.2340908]
[-1.0] [-0.8689798]
[-1.0] [-0.28920925]
[-1.0] [-0.78129876]
[-0.9] [-1.0901178]
[0.3] [-0.1727428]
[0.0] [-0.09089652]
[1.0] [-0.22881286]
[-0.8] [-0.6657192]
[0.0] [0.26457787]
[-0.7] [-0.99340165]
[-1.0] [-0.8794266]
[0.0] [-0.6802235]
[0.0] [-0.22430357]
[0.0] [-0.18013479]
[-0.3] [-0.3695369]
[-1.0] [-0.9112593]
[0.2] [0.12375714]
[0.4] [0.21792276]
[0.0] [-0.9596742]
[0.2] [-0.00103478]
[-0.5] [-0.5141689]
[0.0] [-0.6616202]
[-0.7] [-1.338518]
[0.0] [-0.36309743]
[-0.2] [-0.49410495]
[0.0] [-0.53975874]
[0.5] [0.3721132]
[-0.3] [-0.45480087]
[-0.6] [-0.8768685]
[-0.6] [-1.0018914]
[0.0] [0.09816243]
[0.4] [-

[0.3] [-0.8167463]
[0.4] [-0.66569]
[-1.0] [-0.5336444]
[0.0] [0.4782443]
[0.2] [-0.47011578]
[-0.7] [-0.4390775]
[-0.1] [-1.101729]
[0.0] [-0.5762259]
[-0.3] [-0.42578983]
[-0.4] [-0.28176364]
[-1.0] [-0.58677226]
[-0.8] [-0.8217312]
[-0.9] [-0.7187052]
[-0.8] [-0.6735823]
[-1.0] [-0.7980823]
[-0.9] [-0.9467128]
[0.9] [-0.21561222]
[-0.7] [-0.6228737]
[-0.9] [-0.596354]
[0.4] [-0.57441646]
[0.0] [-0.42399743]
[0.0] [-0.70477694]
[-0.8] [-1.4799808]
[-0.2] [-0.7035226]
[0.0] [-0.17120461]
[0.0] [-0.83140254]
[0.5] [-0.7819043]
[-1.0] [-1.1385156]
[-1.0] [-0.1575544]
[-0.0] [-0.46539325]
[-1.0] [-0.74661064]
[-0.4] [-1.1324073]
[-1.0] [-1.050193]
[-1.0] [0.09016396]
[-0.7] [-0.5779395]
[-1.0] [-0.36936843]
[-0.6] [-1.0473719]
[0.0] [-0.9672488]
[0.2] [-0.15634267]
[-0.9] [-0.11211865]
[-1.0] [-0.5348541]
[-1.0] [-0.85662204]
[-0.9] [-0.9213336]
[0.3] [-0.40855095]
[0.4] [-0.09721804]
[-0.4] [-0.71502477]
[-0.9] [0.22058566]
[0.2] [-0.5335509]
[-0.8] [-0.21276896]
[-1.0] [-0.77276546]
[-

ValueError: Expected 1D or 2D array, got 3D array instead

In [ ]:
def draw_image_with_label(img, label, prediction=None):
    theta = label * 0.69 #Steering range for the car is +- 40 degrees -> 0.69 radians
    line_length = 50
    line_thickness = 3
    label_line_color = (255, 0, 0)
    prediction_line_color = (0, 0, 255)
    pil_image = image.array_to_img(img, K.image_data_format(), scale=True)
    print('Actual Steering Angle = {0}'.format(label))
    draw_image = pil_image.copy()
    image_draw = ImageDraw.Draw(draw_image)
    first_point = (int(img.shape[1]/2),img.shape[0])
    second_point = (int((img.shape[1]/2) + (line_length * math.sin(theta))), int(img.shape[0] - (line_length * math.cos(theta))))
    image_draw.line([first_point, second_point], fill=label_line_color, width=line_thickness)
    
    if (prediction is not None):
        print('Predicted Steering Angle = {0}'.format(prediction))
        print('L1 Error: {0}'.format(abs(prediction-label)))
        theta = prediction * 0.69
        second_point = (int((img.shape[1]/2) + (line_length * math.sin(theta))), int(img.shape[0] - (line_length * math.cos(theta))))
        image_draw.line([first_point, second_point], fill=prediction_line_color, width=line_thickness)
    
    del image_draw
    plt.imshow(draw_image)
    plt.show()